<a href="https://colab.research.google.com/github/punitarani/MAT-494/blob/master/1.4%20Principal%20Component%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.4. Principal Component Analysis
_________________________________
**Used for dimensionality reduction**

### Key Concepts:
* Singular Value Decomposition
* Low-Rank Matrix Approximations
* Principal Component Analysis

In [1]:
import numpy as np
from IPython.display import display, Image

#### Global Variables

In [2]:
# Generate a random mxn matrix
m = 4
n = 5
A = np.random.randint(100, size=(m, n))

print(f"Random {m}x{n} Matrix A:")
print(A)

Random 4x5 Matrix A:
[[23 95 16 66 79]
 [40 98 68 53 65]
 [25 53 35 78 61]
 [54  5 93 52 59]]


## Singular Value Decomposition

Eigendecomposition can only be done for Square matrices.
Singular Value Decomposition (**SVD**) can be used to decompose every matrix, regardless of shape,
or if the eigenvalues are complex or if the eigenvectors are not orthogonal.

SVD Model:
$ A = U \Sigma V ^T $

where:
* $A$: is the Input Matrix. Also referred to using $M$.
* $U$: is an Orthogonal Matrix.
* $\Sigma$: is a Diagonal Matrix. Also referred to using $D$.
* $V$: is an Orthogonal Matrix.

**Time Complexity** is $ O(mn \min\{m,n\}) $, which is computationally expensive.


In [3]:
# Singular Value Decomposition Visualization
svd_url = r"https://upload.wikimedia.org/wikipedia/commons/thumb/c/c8/Singular_value_decomposition_visualisation.svg/1024px-Singular_value_decomposition_visualisation.svg.png"
display(Image(url=svd_url, width=300, unconfined=True))

In [4]:
def svd(A: np.ndarray):
    """
    Singular Value Decomposition
    :param A: Matrix A
    :return: (U, D, V)
    """
    At = A.transpose()

    # Compute the Eigenvalues and Eigenvectors of AAt
    L, U = np.linalg.eig(np.dot(A, At))
    # U is the matrix of eigenvectors

    # D is the squareroot of eigenvalues
    D = np.sqrt(L)
    # Remove 0 elements
    D = D[D != 0]
    # Sort
    D[::-1].sort()

    # Compute the Eigenvalues and Eigenvectors of AtA
    _, V = np.linalg.eig(np.dot(At, A))

    return U, D, V

In [5]:
UA, DA, VA = svd(A)

print(f"Orthogonal Matrix U:\n{UA}\n")
print(f"Diagonal Matrix D:  \n{DA}\n")
print(f"Orthogonal Matrix V:\n{VA}\n")

Orthogonal Matrix U:
[[ 0.5233335   0.53239106 -0.63397587  0.20188217]
 [ 0.57683633  0.08908598  0.31229881 -0.74952851]
 [ 0.45699003  0.06669623  0.63155907  0.62277191]
 [ 0.42958353 -0.83915139 -0.3188686   0.09800891]]

Diagonal Matrix D:  
[256.44308986  89.21431162  39.99564959  13.64171092]

Orthogonal Matrix V:
[[-0.27192148 -0.3120389   0.11191395  0.8657985   0.25798963]
 [-0.51713241  0.65736897  0.5195073   0.05662712 -0.1653652 ]
 [-0.40377039 -0.68521078  0.42069521 -0.35076627 -0.25968102]
 [-0.48001255  0.01598175 -0.68187128  0.10456017 -0.54171141]
 [-0.51496622  0.02699141 -0.27505369 -0.33647661  0.73838352]]



### Verify with np.linalg.svg

In [6]:
UA, DA, VA = np.linalg.svd(A)

print(f"Orthogonal Matrix U:\n{UA}\n")
print(f"Diagonal Matrix D:  \n{DA}\n")
print(f"Orthogonal Matrix V:\n{VA}\n")

Orthogonal Matrix U:
[[-0.5233335  -0.53239106  0.20188217 -0.63397587]
 [-0.57683633 -0.08908598 -0.74952851  0.31229881]
 [-0.45699003 -0.06669623  0.62277191  0.63155907]
 [-0.42958353  0.83915139  0.09800891 -0.3188686 ]]

Diagonal Matrix D:  
[256.44308986  89.21431162  39.99564959  13.64171092]

Orthogonal Matrix V:
[[-0.27192148 -0.51713241 -0.40377039 -0.48001255 -0.51496622]
 [ 0.3120389  -0.65736897  0.68521078 -0.01598175 -0.02699141]
 [-0.11191395 -0.5195073  -0.42069521  0.68187128  0.27505369]
 [-0.25798963  0.1653652   0.25968102  0.54171141 -0.73838352]
 [-0.8657985  -0.05662712  0.35076627 -0.10456017  0.33647661]]



### Performance comparison

In [7]:
%%timeit -n 100
svd(A)

92.8 µs ± 23.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%%timeit -n 100
np.linalg.svd(A)

20.3 µs ± 4.43 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


**np.linalg.svd is ~4x faster than the custom svd implementation.**

## Low-Rank Matrix Approximations
**Applications**:
* Dimensionally reduces matrices and generate a (lossy) compressed version.
* Solve optimization problems
* Matrix completion

#### Ranks
Rank-0: Zero matrix
Rank-1: Rows are multiple of each other
Rank-2: Sum of 2 Rank-1 matrices
Rank-K: Sum of K Rank-1 matrices and cannot be simplified to less than k-1 Rank-1 matrices

### Computing a low-rank approximation
1. Compute SVD
2. Keep k left vectors of U
3. Keep k diagonal values of S
4. Keep k top vectors of V

We essentially retain the k most important values of a vector A
    where $k \le m \le n$

**Choosing the right k value**
The sum of the top k singular values should be at least c times the sum of the other singular values

## Principal Component Analysis
PCA is a dimensionality-reduction method.
It is used to reduces the size and variables of a large data sets while preserving as much information as possible.

### Computing PCA
1. **Standardize Variables**: make variables equally weighted (in terms of range) to prevent one variable from dominating over others.
        $ z = \frac {v - \bar v} {\sigma} $

2. **Compute Covariance Matrix**: compare the relationships between the variables. Helps identify and reduce closely related variables.

3. **Identify the Principal Components**: compute the eigenvectors and eigenvalues of the covariance matrix to determine the Principal Components of the data.

4. **Generate Feature Vector**: Generates the weights for each of the selected Principal Components.

5. **Project Data on the Principal Component Axes**: Remodel and fit the data to the generated Principal Components and Feature Vectors.
